# Atividade Prática Algoritmos Evolutivos
## Aluno: Carlos Eduardo Fontaneli RA: 769949

## O Problema do Caixeiro Viajante
Suponha que um caixeiro viajante tenha que visitar n cidades diferentes, iniciando e
encerrando sua viagem na primeira cidade. Suponha, também, que não importa a
ordem com que as cidades são visitadas e que de cada uma delas o caixeiro pode ir
diretamente para qualquer outra. O problema do caixeiro viajante consiste em descobrir
a rota que torna mínima a viagem total.

### Bibliotecas
Para a resolução dos problemas é necessário importar as seguintes bibliotecas auxiliares:
*random*: para geração de valores aleatórios;
*numpy*: para utilização de funções matemáticas específicas;
*pandas*: para utilização de dataframes para tratamento de informações;
*operator*: para busca de elementos em dicionários;

In [322]:
# Importando as bibliotecas auxiliares necessárias
import random, operator, pandas, numpy

### Cidades

As cidades foram tratadas como classes, pois dessa forma é possível defini-las como um ponto no plano cartesiano (x, y) tornando possível calcular a distância entre elas pelo método Euclidiano. Ademais, definindo-as como classes defini-se para elas métodos próprios de cálculo de distância e representação para cada objeto.

In [323]:
# Declaḿando as cidades como classe
class Cidade:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    # Calcula a distância entre a classe atual e uma dada cidade
    # utiliza o cálculo Euclidiano de distância
    def distancia_entre_cidades(self, cidade):
        distancia_x = abs(self.x - cidade.x)
        distancia_y = abs(self.y - cidade.y)
        distancia = numpy.sqrt((distancia_x ** 2) + (distancia_y ** 2))
        return  distancia

    def __repr__(self):
        return "(" + str(self.x) + "," + str(self.y) + ")"

### Fitness

A função 'fitness' também foi definida como classe para que o objeto podusse receber um percurso(possível trajeto do Caixeiro Viajante) e ser definidas as funções de cálculo de distância do percurso dado e, com a distância, calcular o 'fitness' do trajeto de uma forma que quanto maior o caminho percorrido menor o 'fitness'.

In [324]:
# Definindo a classe Fitness
class Fitness:
    # A classe recebe um percurso para ser calculado seu valor ‘fitness’
    # baseado na distância a ser percorrido pelo percurso
    def __init__(self, percurso):
        self.percurso = percurso
        self.distancia = 0
        self.fitness = 0.0

    def percurso_distancia(self):
        if self.distancia == 0:
            distancia_percorrida = 0
            # Calculando a distância do percurso atual
            for cidade in range(0, len(self.percurso)):
                cidade_atual = self.percurso[cidade]
                cidade_proxima = None
                # Confere se há uma cidade ainda não visitada
                if cidade + 1 < len(self.percurso):
                    cidade_proxima = self.percurso[cidade + 1]
                # Caso todas as cidades tiverem sido visitadas, retorna a cidade inicial
                else:
                    cidade_proxima = self.percurso[0]
                # Calcula a distância entre as cidades e soma à distância do percurso atual
                distancia_percorrida += cidade_atual.distancia_entre_cidades(cidade_proxima)
            self.distancia = distancia_percorrida
        return self.distancia

    # Cálculo do ‘fitness’, quanto maior a distância menor o ‘fitness’ do percurso
    def percurso_fitness(self):
        if self.fitness == 0:
            self.fitness = 1 / float(self.percurso_distancia())
        return self.fitness

### Percursos e População Inicial

Através da biblioteca **random** criou-se um método para gerar amostras de percursos aleatórias baseado na quantidade de cidades a serem percorridas. O percurso é gerado através da escolha de uma sequência aleatória(amostra) das cidades passadas a função.
A população inicial é definida através de uma lista gerada pela função de percurso e contém n(tamanho da população) indivíduos.

In [325]:
# Gerando um percurso
def gera_percurso(cidades):
    percurso = random.sample(cidades, len(cidades))
    return percurso

In [326]:
# Gerador de populacao inicial
def populacao_inicial(populacao_tamanho, cidades):
    populacao = []
    # Gerando individuos como percursos
    for individuo in range(0, populacao_tamanho):
        populacao.append(gera_percurso(cidades))
    return populacao

### Elitismo

A elite é representada por um dicionário ao qual organiza a população por ordem decrescente de 'fitness'. A chave representa o percurso e o valor é o 'fitness' do percurso.

In [327]:
# Definindo os melhores individuos rankeando uma populacao do melhor ao pior
def elitismo(populacao):
    fitnees_individuos = {}
    for individuo in range(0, len(populacao)):
        fitnees_individuos[individuo] = Fitness(populacao[individuo]).percurso_fitness()
    # Ordena do melhor ao piro
    return sorted(fitnees_individuos.items(), key = operator.itemgetter(1), reverse=True)

### Seleção dos Pais
A seleção dos pais é uma combinação entre a elite da população e os demais inviduos. Fora a elite, os individuos são selecionados através de uma probabilidade de seleção relacionada ao seu valor 'fitness' em relação ao 'fitness' da população.

In [328]:
# Selecionando os pais
def selecao(elitismo, tamanho_elite):
    individuos_selecionados = []
    # Transformando a população ordenada em um 'dataframe' para cálculos do 'fitness'
    # relativo à população
    df = pandas.DataFrame(numpy.array(elitismo), columns=['Individuo', 'Fitness'])
    df['cum_sum'] = df.Fitness.cumsum()
    # 'fitness' da população baseado na porcentagem do 'fitness' ao longo do percurso
    # pelo 'fitness' total da população
    df['cum_perc'] = 100 * df.cum_sum / df.Fitness.sum()

    # Selecionando a elite
    for individuo in range(0, tamanho_elite):
        individuos_selecionados.append(elitismo[individuo][0])
    # Selecionando os demais indivíduos
    for individuo in range(0, len(elitismo) - tamanho_elite):
        # Probabilidade de seleção
        escolhido = 100 * random.random()
        for individuo in range(0, len(elitismo)):
            if escolhido <= df.iat[individuo, 3]:
                individuos_selecionados.append(elitismo[individuo][0])
                break
    return individuos_selecionados

### Pais, Cruzamento e Cruzamento de Populações

Através do uso da seleção os pais são retirados da população inicial.
O cruzamento é feito por meio do 'ordered crossover' onde uma porção, selecionada aleatoriamente, dos genes do primeiro pai é selecionado e tranferido para o filho, o qual tem seu cromossomo restante completado com os genes do segundo pai.
Por fim, o cruzamento é generalizado para toda uma população por meio da função que recebe um grupo de pais e o tamanho da elite. A função reserva a elite para a geração futura e gera os demais filhos por meio do cruzamento entre os pais.


In [329]:
# Gerando os pais que Irã fazer parte do cruzamento
def pais(populacao, individuos_selecionados):
    cruzamento = []
    for individuo in range(0, len(individuos_selecionados)):
        cruzamento.append(populacao[individuos_selecionados[individuo]])
    return cruzamento

In [330]:
# Usando a técnica de 'ordered crossover'
def cruzamento(pai1, pai2):
    filho = []
    genes_pai1 = []
    genes_pai2 = []

    # Gerando aleatoriamente começo e fim do crossover dos genes relativos ao pai1
    gene_a = int(random.random() * len(pai1))
    gene_b = int(random.random() * len(pai1))
    # Verificando quais são os genes mínimos e máximos para definição do começo e do fim
    comeco_pai1 = min(gene_a, gene_b)
    fim_pai1 = max(gene_a, gene_b)

    # Separando os genes do pai1
    for gene in range(comeco_pai1, fim_pai1):
        genes_pai1.append(pai1[gene])

    # Separando do pai2 os genes do pai1 que não estão presentes no filho
    genes_pai2 = [gene for gene in pai2 if gene not in genes_pai1]

    filho = genes_pai1 + genes_pai2

    return filho

In [331]:
# Generalizando a função de cruzamento para populações inteiras
def cruzamento_populacao(pais, tamanho_elite):
    filhos = []
    tamanho = len(pais) - tamanho_elite
    pais_selecionados = random.sample(pais, len(pais))

    # Separando a elite
    for elitista in range(0, tamanho_elite):
        filhos.append(pais[elitista])

    # Gerando o resto da populacao por cruzamento
    for individuo in range(0, tamanho):
        filho = cruzamento(pais_selecionados[individuo], pais_selecionados[len(pais)-individuo-1])
        filhos.append(filho)

    return filhos

### Mutação e Mutação de Populações

A mutacação é definida por uma proporção de mutantes em cada população. A mutação genética acontece na troca de um indivíduo por outro durante o percurso.
Generaliza-se a mutacação para populações de uma forma onde uma população é passada para função e, aplicando a método de mutação nela, retorna-se uma população nova onde pode ou não conter percursos mutados.

In [332]:
# Definindo mutacação da população através da inversao de individuos em um dado percurso
def mutacao(percurso, proporcao_mutantes):
    for individuo1 in range(len(percurso)):
        # Aplicando mutacao condicionado a proporcao passada pra funcao
        if random.random() < proporcao_mutantes:
            # Escolhendo aleatoriamente um individuo a ser invertido
            individuo2 = int(random.random() * len(percurso))

            # Realizando inversão
            cidade1 = percurso[individuo1]
            cidade2 = percurso[individuo2]
            percurso[individuo1] = cidade2
            percurso[individuo2] = cidade1

    return percurso

In [333]:
# Generalizando a mutacação para populações
def mutacao_populacao(populacao, proporcao_mutantes):
    populacao_mutante = []

    # Aplicando mutacao em cada indivíduo da população
    for percurso in range(0, len(populacao)):
        percuso_mutante = mutacao(populacao[percurso], proporcao_mutantes)
        populacao_mutante.append(percuso_mutante)

    return populacao_mutante

### Gerações

As novas gerações são definidas através da aplicação dos métodos definidos anteriormente na população atual. Dessa forma, cada população passa pela ordenação da população(elitismo), seleção da elite, definição dos demais pais, geração dos filhos por meio do cruzamento e, por fim, definição da próxima geração após aplicada a mutação nos filhos.

In [334]:
# Criações de gerações
def geracoes(populacao_atual, tamanho_elite, proporcao_mutantes):
    # Definindo a elite
    ordenacao_populacao = elitismo(populacao_atual)
    # Selecionando os melhores individuos
    elite = selecao(ordenacao_populacao, tamanho_elite)
    # Selecionando os pais para o cruzamentos
    pais_selecionados = pais(populacao_atual, elite)
    # Gerando os filhos por meio do cruzamento dos pais
    filhos = cruzamento_populacao(pais_selecionados, tamanho_elite)
    # Definindo a proxima geracao aplicando mutação sobre os filhos
    proxima_geracao = mutacao_populacao(filhos, proporcao_mutantes)

    return proxima_geracao

### Algoritmo Genético

O algoritmo genético por fim é a aplicação dos métodos para buscar o melhor percurso. Assim sendo, ele recebe os genes iniciais a serem evoluídos, a definição do tamanho das populações a serem geradas, o tamanho da elite, a proporção de mutantes e a quantidade de futuras gerações.
O algoritmo executa inicialmente exibindo o melhor indivíduo da população inicial gerada, ou seja, o melhor percurso inicial e sua distância. Após a execução de todas as gerações é exibido o indivíduo gerado, isto é, o melhor percurso e sua distância.

In [335]:
# Algortimo Genético
def algoritmo_genetico(genes_iniciais, tamanho_populacao, tamanho_elite, proporcao_mutantes, numero_geracoes):
    populacaos= populacao_inicial(tamanho_populacao, genes_iniciais)

    # Adicionando a cidade inicial ao final da representação do percurso
    melhor_inicial =  populacaos[elitismo(populacaos)[0][0]]
    melhor_inicial.append(melhor_inicial[0])
    print("Melhor percurso inicial: ", melhor_inicial)
    print("Distancia do melhor percurso inicial: " + str(1 / elitismo(populacaos)[0][1]))


    for individuo in range(0, numero_geracoes):
        populacaos = geracoes(populacaos, tamanho_elite, proporcao_mutantes)

    print()

    # Adicionando a cidade inicial ao final da representação do percurso
    melhor_final = populacaos[elitismo(populacaos)[0][0]]
    melhor_final.append(melhor_final[0])
    print("Melhor percurso final: ",  melhor_final)
    print("Distância distância do melhor percurso final: " + str(1 / elitismo(populacaos)[0][1]))

    return melhor_final

### Execução do Algoritmo Genético

In [336]:
# Executando GA

# Criando as cidades
cidades = []
for i in range(0, 6):
    cidades.append(Cidade(x=int(random.random() * 1000), y=int(random.random() * 1000)))

# Aplicando GA nas cidades
melhor_percurso = algoritmo_genetico(genes_iniciais=cidades, tamanho_populacao=10, tamanho_elite=3, proporcao_mutantes=0.01, numero_geracoes=100)

Melhor percurso inicial:  [(666,74), (446,309), (60,488), (189,855), (954,890), (892,250), (666,74)]
Distancia do melhor percurso inicial: 2831.647919630869

Melhor percurso final:  [(666,74), (446,309), (60,488), (189,855), (954,890), (892,250), (666,74)]
Distância distância do melhor percurso final: 2831.647919630869
